## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from pymystem3 import Mystem

data = pd.read_csv('/datasets/data.csv')
data.info()
data.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выполним поиск нулевых значений, полных дубликатов, уникальных значений, минимумов и максимумов в каждом столбце.

In [2]:
data[(data['days_employed'].isnull()) & (data['total_income'].isnull())].shape[0]

2174

In [3]:
data.duplicated().sum()

54

In [4]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [5]:
np.sort(data['dob_years'].unique())

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

In [6]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [7]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [8]:
data.min()

children                               -1
days_employed                    -18388.9
dob_years                               0
education                          ВЫСШЕЕ
education_id                            0
family_status       Не женат / не замужем
family_status_id                        0
gender                                  F
income_type                   безработный
debt                                    0
total_income                      20667.3
purpose                        автомобили
dtype: object

In [9]:
data.max()

children                         20
days_employed                401755
dob_years                        75
education            ученая степень
education_id                      4
family_status       женат / замужем
family_status_id                  4
gender                          XNA
income_type                 студент
debt                              1
total_income             2.2656e+06
purpose             сыграть свадьбу
dtype: object

### Вывод

В результате предварительного анализа таблицы можно сделать следующен выводы:
    1. В таблице 21525 наблюдений. В стобцах 'days_employed' и 'total_income' по 2174 пропуска. В наблюдениях, имеющих пропуск в одном из этих столбцов, обязательно присутствует пропуск в другом. Можно сделать вывод о закономерности данных пропусков.
    2. В столбцах 'days_employed' и 'total_income' тип данных 'float64'. В столбце 'days_employed' присутствуют отрицательные и аномальные значения.
    3. В столбце 'education' одни и те же значения записаны в разных регистрах.
    4. В столбце 'purpose' некоторые выражения имеют один и тот же смысл, но записаны по-разному.
    5. Обнаружено 54 полных дубликата.
    6. В столбцах 'days_employed', 'children', 'dob_years' присутствуют аномальные значения.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [10]:
data[(data['days_employed'].isnull()) & (data['total_income'].isnull())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


В таблице 2174 строк с пропусками в столбцах 'days_employed' и 'total_income', что составляет 10% от всей выборки, поэтому
удаление строк с пропусками нецелесообразно. Пропуски в столбце 'total_income' заполним медианными значениями дохода для
каждого типа занятости.

In [11]:
pensionary_salary_median = data[data['income_type'] == 'пенсионер']['total_income'].median()
pensionary_salary_median

118514.48641164352

In [12]:
data.loc[(data['income_type'] == 'пенсионер') & (data['total_income'].isnull()), 'total_income'] = pensionary_salary_median
data[(data['income_type'] == 'пенсионер') & (data['total_income'].isnull())].shape[0]

0

In [13]:
сivil_servant_salary_median = data[data['income_type'] == 'госслужащий']['total_income'].median()
сivil_servant_salary_median

150447.9352830068

In [14]:
data.loc[(data['income_type'] == 'госслужащий') & (data['total_income'].isnull()), 'total_income'] = сivil_servant_salary_median
data[(data['income_type'] == 'госслужащий') & (data['total_income'].isnull())].shape[0]

0

In [15]:
employee_salary_median = data[data['income_type'] == 'сотрудник']['total_income'].median()
employee_salary_median

142594.39684740017

In [16]:
data.loc[(data['income_type'] == 'сотрудник') & (data['total_income'].isnull()), 'total_income'] = employee_salary_median
data[(data['income_type'] == 'сотрудник') & (data['total_income'].isnull())].shape[0]

0

In [17]:
partner_salary_median = data[data['income_type'] == 'компаньон']['total_income'].median()
partner_salary_median

172357.95096577113

In [18]:
data.loc[(data['income_type'] == 'компаньон') & (data['total_income'].isnull()), 'total_income'] = partner_salary_median
data[(data['income_type'] == 'компаньон') & (data['total_income'].isnull())].shape[0]

0

In [19]:
unemployed_salary_median = data[data['income_type'] == 'безработный']['total_income'].median()
unemployed_salary_median

131339.7516762103

In [20]:
data.loc[(data['income_type'] == 'безработный') & (data['total_income'].isnull()), 'total_income'] = unemployed_salary_median
data[(data['income_type'] == 'безработный') & (data['total_income'].isnull())].shape[0]

0

In [21]:
businessman_salary_median = data[data['income_type'] == 'предприниматель']['total_income'].median()
businessman_salary_median

499163.1449470857

In [22]:
data.loc[(data['income_type'] == 'предприниматель') & (data['total_income'].isnull()), 'total_income'] = businessman_salary_median
data[(data['income_type'] == 'предприниматель') & (data['total_income'].isnull())].shape[0]

0

In [23]:
student_salary_median = data[data['income_type'] == 'студент']['total_income'].median()
student_salary_median

98201.62531401133

In [24]:
data.loc[(data['income_type'] == 'студент') & (data['total_income'].isnull()), 'total_income'] = student_salary_median
data[(data['income_type'] == 'студент') & (data['total_income'].isnull())].shape[0]

0

In [25]:
decree_salary_median = data[data['income_type'] == 'в декрете']['total_income'].median()
decree_salary_median

53829.13072905995

In [26]:
data.loc[(data['income_type'] == 'в декрете') & (data['total_income'].isnull()), 'total_income'] = decree_salary_median
data[(data['income_type'] == 'в декрете') & (data['total_income'].isnull())].shape[0]

0

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропуски в столбце 'total_income' успешно обработаны. 
Прежде, чем приступить к обработке пропусков в столбце 'days_employed', требуется решить проблему с отрицательными и теоретически невозможными значениями.

In [29]:
data[data['days_employed'] < 0]['days_employed'].count()

15906

In [30]:
data[data['days_employed'] < 0]['days_employed'].max()

-24.14163324048118

In [31]:
data[data['days_employed'] < 0]['days_employed'].min()

-18388.949900568383

Среди отрицательных значений аномалии отсутствуют, следовательно, можно выдвинуть гипотезу, что отрицательные значения были получены в результате ошибки при переносе данных. Возможно, предполагался дефис. В данной таблице 15906 отрицательных значений, что составляет 73.9%.
Игнорировать такой большой объём данных мы не можем, поэтому приведём отрицательные значения к абсолютным значениям.

In [32]:
data['days_employed'] = data['days_employed'].abs()
data[data['days_employed'] < 0]['days_employed'].count()

0

Напишем функцию 'mistake', которая принимает строку таблицы в качестве аргумента, создаёт столбец 'employed_mistake', в котором указывается: возможно ли в данном возрасте иметь такой трудовой стаж. Согласно Трудовому кодексу РФ, официально трудоустроиться
может лицо, достигшее 14 лет. Поэтому максимально возможный стаж в днях : (возраст - 14) * 365. Прежде, чем приступить к созданию функции, необходимо избавиться от аномальных значений в столбце 'dob_years'.

In [33]:
data[data['dob_years'] == 0]['income_type'].value_counts()

сотрудник      55
компаньон      20
пенсионер      20
госслужащий     6
Name: income_type, dtype: int64

В таблице 101 наблюдение, где значение признака 'dob_years' равно 0, что составляет 0.47%. В данных аномалиях не прослеживается закономерность. Можно выдвинуть гипотезу, что аномалии были допущены в результате утери данных или недостатков системы переноса данных. Удалим строки со значением стобца 'dob_years' = 0.

In [34]:
data = data[data['dob_years'] != 0]
data.reset_index(drop=True)
data[data['dob_years'] == 0].shape[0]

0

Строки, признак 'dob_years' которых равен 0, удалены. Теперь можем приступить к написанию функции 'mistake'.

In [35]:
def mistake(row):
    age = row['dob_years']
    days_employed = row['days_employed']
    max_days = (age - 14) * 365
    if days_employed > max_days:
        return 1
    else:
        return 0
data['employed_mistake'] = data.apply(mistake, axis=1)
data[data['employed_mistake'] == 1]['income_type'].value_counts()

пенсионер      3426
сотрудник        10
компаньон         5
безработный       2
госслужащий       1
Name: income_type, dtype: int64

In [36]:
data[data['employed_mistake'] == 0]['income_type'].value_counts()

сотрудник          11054
компаньон           5060
госслужащий         1452
пенсионер            410
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [37]:
data[data['income_type'] == 'пенсионер'].shape[0]

3836

В таблице 3836 наблюдений, связанных с пенсионерами. В стобце 'days_employed' у 3426 пенсионеров теоретически невозможное значение. В оставшихся 410 случаях значение вовсе отсутствует. Можно сделать вывод о закономерности некорректных данных о стаже пенсионеров. 
Вычислим медианные значения трудового стажа для каждого возраста. Заменим ими отсутствующие и аномальные значения в стобце 'days_employed'. Данное решение проблемы является грубым, но в рамках нашего исследования трудовой стаж не играет роли.

In [38]:
age_list = np.sort(data['dob_years'].unique()) # отсортированный список всех возрастов
age_dict = dict.fromkeys(age_list, 0) # создаём словарь из списка возрастов
for age in age_dict:
    age_data = data[(data['dob_years'] == age) & (data['income_type'] != 'пенсионер')
                    & (data['employed_mistake'] == 0) & (data['days_employed'] > 0)]
    age_median = age_data['days_employed'].median()
    age_dict[age] = age_median
age_dict

{19: 724.4926104717459,
 20: 674.8389792229723,
 21: 618.7338169028955,
 22: 699.0612136526936,
 23: 690.2042083531062,
 24: 947.7310428833248,
 25: 919.1993879922846,
 26: 1072.9735666505499,
 27: 1150.6805177028941,
 28: 1136.7888256136375,
 29: 1313.5564950674602,
 30: 1420.586862991635,
 31: 1302.524254878967,
 32: 1435.782308209095,
 33: 1424.2833466420857,
 34: 1600.2341543269727,
 35: 1607.5093900098022,
 36: 1785.6200361136218,
 37: 1799.9539042083331,
 38: 1778.0803346622388,
 39: 1859.2429671443758,
 40: 1698.9245182189593,
 41: 1858.5569092545265,
 42: 2211.493070889809,
 43: 1762.8803925063933,
 44: 1903.3406623942617,
 45: 2166.1491959024506,
 46: 2027.6302309683472,
 47: 2031.1673595987013,
 48: 2229.7814080504213,
 49: 2376.1948282138983,
 50: 2191.8081570672593,
 51: 2062.3432879466927,
 52: 2338.7210419711487,
 53: 2249.036018268497,
 54: 2100.650584044376,
 55: 2646.5505856625814,
 56: 2255.2144649999373,
 57: 1940.1682509483298,
 58: 2271.0033744946068,
 59: 2725.756

In [39]:
for age, days_employed in age_dict.items():
    data.loc[((data['employed_mistake'] == 1) | (data['days_employed'].isnull()))
             & (data['dob_years'] == age), 'days_employed'] = days_employed 

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 13 columns):
children            21424 non-null int64
days_employed       21424 non-null float64
dob_years           21424 non-null int64
education           21424 non-null object
education_id        21424 non-null int64
family_status       21424 non-null object
family_status_id    21424 non-null int64
gender              21424 non-null object
income_type         21424 non-null object
debt                21424 non-null int64
total_income        21424 non-null float64
purpose             21424 non-null object
employed_mistake    21424 non-null int64
dtypes: float64(2), int64(6), object(5)
memory usage: 2.3+ MB


In [41]:
data['days_employed'].max()

16593.472817263817

In [42]:
data['days_employed'].min()

24.14163324048118

Пропуски и аномальные значения в стобце 'days_employed' успешно обработаны.

In [43]:
data = data.drop('employed_mistake', 1)

### Вывод

В разделе "Обработка пропусков" была выполнена следующая работа:
    1. Обработаны пропуски в столбце 'total_income' с помощью замены на медианные значения для каждого типа занятости.
    2. Обработаны аномальные значения, равные 0, в столбце 'dob_years'
    3. Обработаны аномальные и пропущенные значения в столбце 'days_employed' с помощью замены на медианные значения для каждого возраста.

### Замена типа данных

Столбцы 'days_employed' и 'total_income' имеют тип данных float64. Числа с плавающей точной усложняют визуальную оценку данных таблицы. В нашем случае приведение чисел к типу int64 не повлияет на результат исследования. 
В этом разделе заменим тип данных float64 в столбцаъ 'days_employed' и 'total_income' на int64.

In [44]:
data['days_employed'] = data['days_employed'].astype(int)
data['total_income'] = data['total_income'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
children            21424 non-null int64
days_employed       21424 non-null int64
dob_years           21424 non-null int64
education           21424 non-null object
education_id        21424 non-null int64
family_status       21424 non-null object
family_status_id    21424 non-null int64
gender              21424 non-null object
income_type         21424 non-null object
debt                21424 non-null int64
total_income        21424 non-null int64
purpose             21424 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

Ввиду того, что приведение чисел с плавающей точкой к целочисленному типу не искажает результатов наблюдения, столбцы 'days_employed' и 'total_income' были приведены к типу int64.

### Обработка дубликатов

In [45]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Прежде, чем обработать дубликаты, мы должны привести столбец 'education' к единому регистру.

In [46]:
data['education'] = data['education'].str.lower()

In [47]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [48]:
data.duplicated().sum()

71

В таблице 71 полных дубликатов. Избавимся от них.

In [49]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

Полные дубликаты успешно удалены. Теперь выполним поиск дубликатов по некоторым столбцам.

In [50]:
data.duplicated(subset=['total_income', 'education', 'dob_years', 'days_employed', 'children', 
                        'income_type', 'debt', 'family_status', 'gender']).sum()

674

In [51]:
data = data.drop_duplicates(subset=['total_income', 'education', 'dob_years', 'days_employed', 'children', 
                             'income_type', 'debt', 'family_status', 'gender']).reset_index(drop=True)
data.duplicated(subset=['total_income', 'education', 'dob_years', 'days_employed', 'children', 
                        'income_type', 'debt', 'family_status', 'gender']).sum()

0

### Вывод

В данном разделе были удалены полные дубликаты и дубликаты без учёта столбца 'purpose', т.к. данный столбец должен быть обработан с помощью лемматизации и категоризации.

### Лемматизация

Выведем на экран все уникальные значения столбца 'purpose'.

In [52]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [53]:
Counter(data['purpose']) 

Counter({'покупка жилья': 622,
         'приобретение автомобиля': 444,
         'дополнительное образование': 442,
         'сыграть свадьбу': 750,
         'операции с жильем': 625,
         'образование': 427,
         'на проведение свадьбы': 751,
         'покупка жилья для семьи': 615,
         'покупка недвижимости': 597,
         'покупка коммерческой недвижимости': 635,
         'покупка жилой недвижимости': 582,
         'строительство собственной недвижимости': 613,
         'недвижимость': 611,
         'строительство недвижимости': 593,
         'на покупку подержанного автомобиля': 465,
         'на покупку своего автомобиля': 486,
         'операции с коммерческой недвижимостью': 630,
         'строительство жилой недвижимости': 600,
         'жилье': 622,
         'операции со своей недвижимостью': 607,
         'автомобили': 462,
         'заняться образованием': 385,
         'сделка с подержанным автомобилем': 473,
         'получение образования': 426,
         'авт

Среди уникальных значений столбца 'purpose' можно выделить 4 категории целей кредита: недвижимость, образование, автомобиля, свадьба.
Создадим словарь, ключами которого являются цели кредита, а значениями - выделенные категории.

In [54]:
purpose_array = data['purpose'].unique()
category_purpose = {}
m = Mystem()
for purpose in purpose_array:
    lemmas = m.lemmatize(purpose)
    if 'автомобиль' in lemmas:
        category_purpose[purpose] = 'автомобиль'
    elif 'недвижимость' in lemmas or 'жилье' in lemmas:
        category_purpose[purpose] = 'недвижимость'
    elif 'образование' in lemmas:
        category_purpose[purpose] = 'образование'
    elif 'свадьба' in lemmas:
        category_purpose[purpose] = 'свадьба'
category_purpose

{'покупка жилья': 'недвижимость',
 'приобретение автомобиля': 'автомобиль',
 'дополнительное образование': 'образование',
 'сыграть свадьбу': 'свадьба',
 'операции с жильем': 'недвижимость',
 'образование': 'образование',
 'на проведение свадьбы': 'свадьба',
 'покупка жилья для семьи': 'недвижимость',
 'покупка недвижимости': 'недвижимость',
 'покупка коммерческой недвижимости': 'недвижимость',
 'покупка жилой недвижимости': 'недвижимость',
 'строительство собственной недвижимости': 'недвижимость',
 'недвижимость': 'недвижимость',
 'строительство недвижимости': 'недвижимость',
 'на покупку подержанного автомобиля': 'автомобиль',
 'на покупку своего автомобиля': 'автомобиль',
 'операции с коммерческой недвижимостью': 'недвижимость',
 'строительство жилой недвижимости': 'недвижимость',
 'жилье': 'недвижимость',
 'операции со своей недвижимостью': 'недвижимость',
 'автомобили': 'автомобиль',
 'заняться образованием': 'образование',
 'сделка с подержанным автомобилем': 'автомобиль',
 'полу

### Вывод

В данном разделе, применив лемматизацию, мы создали словарь, показывающий: к какой категории относится каждая цель кредита. 
Это действие необходимо для категоризации данных, которая будет выполнена в следующем разделе.

### Категоризация данных

Создадим столбец 'category_purpose', который содержит категории целей получения кредита.

In [55]:
data['category_purpose'] = data['purpose'].map(category_purpose)

In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20679 entries, 0 to 20678
Data columns (total 13 columns):
children            20679 non-null int64
days_employed       20679 non-null int64
dob_years           20679 non-null int64
education           20679 non-null object
education_id        20679 non-null int64
family_status       20679 non-null object
family_status_id    20679 non-null int64
gender              20679 non-null object
income_type         20679 non-null object
debt                20679 non-null int64
total_income        20679 non-null int64
purpose             20679 non-null object
category_purpose    20679 non-null object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


In [57]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Для ответа в следующем шаге на вопрос: 'Есть ли зависимость между семейным положением и возвратом кредита в срок?' будет рациональнее рассматривать не каждый вид семейного положения в отдельности, а разбить на 2 категории по признаку наличия супруга или сожителя.
Создадим столбец 'partner'. Для этого напишем функцию, принимающую в качестве аргумента семейное положение. Если человек женат/замужем или состоит в граждаском браке, то функция возвращает '1', в ином случае - '0'.

In [58]:
def family_happiness(family_status):
    if family_status == 'женат / замужем' or family_status == 'гражданский брак':
        return 1
    return 0
data['partner'] = data['family_status'].apply(family_happiness)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20679 entries, 0 to 20678
Data columns (total 14 columns):
children            20679 non-null int64
days_employed       20679 non-null int64
dob_years           20679 non-null int64
education           20679 non-null object
education_id        20679 non-null int64
family_status       20679 non-null object
family_status_id    20679 non-null int64
gender              20679 non-null object
income_type         20679 non-null object
debt                20679 non-null int64
total_income        20679 non-null int64
purpose             20679 non-null object
category_purpose    20679 non-null object
partner             20679 non-null int64
dtypes: int64(8), object(6)
memory usage: 2.2+ MB


Для ответа в следующем шаге на вопрос: 'Есть ли зависимость между уровнем дохода и возвратом кредита в срок?' разобьём клиентов банка на несколько категорий по уровню дохода: низкий, средний, выше среднего, высокий. Для того, чтобы точно определить пороги для категорий доходов, вычислим квартили для столбца 'total_income'.

In [59]:
data['total_income'].quantile([0.25, 0.5, 0.75]) # исправление к оранжевому комментарию ниже

0.25    106034.0
0.50    143369.0
0.75    198096.5
Name: total_income, dtype: float64

In [60]:
def income_category(total_income):
    if total_income < 106034:
        return 'низкий'
    elif 106034 <= total_income < 143369:
        return 'средний'
    elif 143369 <= total_income < 198096:
        return 'выше среднего'
    return 'высокий'
data['income_category'] = data['total_income'].apply(income_category)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20679 entries, 0 to 20678
Data columns (total 15 columns):
children            20679 non-null int64
days_employed       20679 non-null int64
dob_years           20679 non-null int64
education           20679 non-null object
education_id        20679 non-null int64
family_status       20679 non-null object
family_status_id    20679 non-null int64
gender              20679 non-null object
income_type         20679 non-null object
debt                20679 non-null int64
total_income        20679 non-null int64
purpose             20679 non-null object
category_purpose    20679 non-null object
partner             20679 non-null int64
income_category     20679 non-null object
dtypes: int64(8), object(7)
memory usage: 2.4+ MB


In [62]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category_purpose,partner,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,1,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,1,выше среднего
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,1,высокий
4,0,2249,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,1,выше среднего
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,1,высокий
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,1,высокий
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,1,средний
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,1,низкий
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,1,выше среднего


### Вывод

В данном разделе были категоризированы цели кредита, доход и семейное положение, что необходимо для формирования ответов на вопросы проекта.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [63]:
data['children'].value_counts()

 0     13513
 1      4676
 2      1993
 3       326
 20       75
-1        47
 4        40
 5         9
Name: children, dtype: int64

В столбце 'children' 2 аномальных значения. Предположительно, значение -1 получено в результате случайно поставленного дефиса. А значение 20 - в результате случайно поставленного нуля. Заменим их на значения 1 и 2 соответственно.

In [64]:
data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 2
data['children'].value_counts()

0    13513
1     4723
2     2068
3      326
4       40
5        9
Name: children, dtype: int64

Сгруппируем таблицу по столбцу 'children' и применим функции 'count', 'sum', 'mean' к столбцу 'debt'. 

In [65]:
(data
     .groupby('children')
     .agg({'debt': ['count', 'sum', 'mean']})
)

debt                
          count   sum      mean
children                       
0         13513  1051  0.077777
1          4723   441  0.093373
2          2068   201  0.097195
3           326    27  0.082822
4            40     4  0.100000
5             9     0  0.000000

### Вывод

Из данной таблицы можно сделать следующий вывод. 
Количество должников возрастает с количеством детей до 2-ух детей включительно. У клиентов с 3-5 детьми не наблюдается стабильнсти в росте кол-ва должников. 
Имеет место гипотеза, что нулевой процент должников по кредиту среди клиентов с 5 детьми и отсутствие роста кол-ва должников при возрастании кол-ва детей, начиная с клиентов, имеющих от 3 детей, связаны с малым кол-вом клиентов с данным кол-вом детей по сравнению с клиентами, имеющими 0-2 детей. Например,  4723 клиента с 1 ребёнком против 326 клиентов с 3 детьми. В первом случае точность оценки намного выше, так как объём выборки больше.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Сгруппируем таблицу по столбцу 'family_status' и применим функции 'count', 'sum', 'mean' к столбцу 'debt'.

In [67]:
(data
     .groupby('family_status')
     .agg({'debt': ['count', 'sum', 'mean']})
)

debt               
                       count  sum      mean
family_status                              
Не женат / не замужем   2739  273  0.099671
в разводе               1173   85  0.072464
вдовец / вдова           925   62  0.067027
гражданский брак        4050  385  0.095062
женат / замужем        11792  919  0.077934

### Вывод

По данной таблице можно сделать следующие выводы::
1. Кол-во должников "в разводе" практически равно кол-ву должников "женат/замужем". Имеет место гипотеза, что это связано с положением СК РФ, гласящем о том, что кредиты и долги супругов делятся пополам после развода.
2. Кол-во должников "гражданский брак" практически равно кол-ву должников "Не женат / не замужем". Предположительно, это связано с тем, что в обоих случаях клиенты не состоят в зарегистрированных отношениях. Кол-во должников в этих категориях выше, чем кол-во должников в категориях, рассмотренных в предыдущем пункте. Имеет место гипотеза, что в данных случаях кредит выплачивает 1 человек, поэтому и вероятность задолженности выше.
3. Кол-во должников среди вдовцов ниже, чем у остальных категорий. Возможно, это связано с тем, что при смерти супруга застрахованные кредиты покрывается за счёт страховки, а в ином случае распределяются между родственниками пропорционально наследству.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сгруппируем таблицу по столбцу 'income_category' и применим функции 'count', 'sum', 'mean' к столбцу 'debt'.

In [68]:
(data
     .groupby('income_category')
     .agg({'debt': ['count', 'sum', 'mean']})
)

debt               
                count  sum      mean
income_category                     
высокий          5170  366  0.070793
выше среднего    5170  451  0.087234
низкий           5170  407  0.078723
средний          5169  500  0.096731

### Вывод

По данной таблице можно сделать следующие выводы:
1. Реже всех задолженности по кредиту имеют клиенты с высоким уровнем дохода. Предположительно, это связано с большим количеством свободных денег.
2. Немного "проигрывают" рассмотренным выше клиенты с низким уровнем дохода. Имеет место гипотеза, что это связаном с нежеланием людей брать дорогие кредиты из-за низкого уровня дохода.
3. Чаще всех имеют задолженности по кредиту люди со средним уровнем дохода. Предположительно, клиенты со средним уровнем дохода переоценивают свою платёжеспособность и берут дорогие кредиты.

- Как разные цели кредита влияют на его возврат в срок?

Сгруппируем таблицу по столбцу 'category_purpose' и применим функции 'count', 'sum', 'mean' к столбцу 'debt'.

In [69]:
(data
     .groupby('category_purpose')
     .agg({'debt': ['count', 'sum', 'mean']})
)

debt               
                  count  sum      mean
category_purpose                      
автомобиль         4151  397  0.095640
недвижимость      10418  776  0.074486
образование        3836  367  0.095673
свадьба            2274  184  0.080915

### Вывод

Из этой таблицы можно сделать следующие выводы:
1. Наибольшее и равное кол-во задолжников имеют клиенты категории 'образование' и 'автомобиль'. Как правило, средний срок автокредита составляет 3-5 лет, а кредит на образование часто выдаётся на срок обучения. Срок кредита мал, а стоимость, как правило, высока, поэтому кол-во должников в данных категориях выше, чем в остальных. К тому же, часто люди легкомысленно относятся к данным категориям кредитов, недостаточно оценив свои возможности.
2. Наименьшее кол-во должников в категории "недвижимость". Имеет место гипотеза, что это связано с большим сроком кредита и серьёзной оценкой клиента своей платёжеспособности перед принятием решения об оформлении кредита.

### Шаг 4. Общий вывод

Из данного исследования можно сделать следующие выводы:
1. Кол-во детей напрямую влияет на вероятность появления задолженности. Проблема заключается в сложности оценки платёжеспособности многодетных семей из-за их малого кол-ва относительно семей с 0-2 детьми. Меньше всех задолженностей имеют клиенты без детей.
2. Вероятность возникновения задолженности по кредиту выше всех у клиентов категорий  "Не женат/не замужем" и "гражданский брак". Ниже всех - "вдовец / вдова"
3. Меньше всего должников по уровню дохода в категории "высокий". Больше всех - "выше среднего".
4. Наибольшее кол-во должников по цели получения кредита имеют категории: "образование" и "автомобиль". Меньше всех = "недвижимость".
Стоит отметить, что для большей объективности следует оценивать платёжеспособность клиентов по нескольким показателям одновременно, а не по одному, как в нашем проекте.
Портрет самого надёжного клиента, исходя из результатов исследования: клиент без детей, вдова/вдовец, с выскоим доходом, оформил кредит на недвижимость.
Портрет самого рискованного клиента: у клиента больше 2 детей, не состоит в браке, со средним доходом, оформил кредит на автомобиль или образование.